In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from time import time


In [5]:
from kNN import kNN, kNN_with_pq_NCC, kNN_with_pq_SINK, MAP, avg_recall_measure
from time import time
from Causal_inference import check_with_original
import Representation

Importing datasets

In [8]:
import os
cwd = os.getcwd()
print(cwd)

/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine


In [14]:
dataset_names = ["FaceFour", "InlineSkate", "PickupGestureWiimoteZ", "SemgHandMovementCh2"]

to_import =  ["mixsd0.1_0.1_causaldb", "mixsd0.1_0.05_causaldb", "mixsd0.2_0.1_causaldb", "mixsd0.2_0.05_causaldb", "randomsd0.1_effectdb", "randomsd0.2_effectdb", "rwalksd0.1_effectdb", "rwalksd0.05_effectdb"]

path = cwd +"/test_files/"
dataset_dict = {}

counter = 0
for each in dataset_names:
    dataset_dict[each] = {}
    dataset_dict[each]["truemat"] = np.load(str(path+each+"_split_truemat.npy"))
    print(dataset_dict[each]["truemat"])
    dataset_dict[each]["causaldb"] = np.load(str(path+ each+"_causaldb.npy"))
    for import_type in to_import:
        dataset_dict[each][import_type] = np.load(str(path+each+"_"+import_type+".npy"))
        # if counter == 0:
        #     print(dataset_dict[each][import_type].shape)
        #     # counter += 1



(112, 350)
(112, 350)
(112, 350)
(112, 350)
(112, 350)
(112, 350)
(112, 350)
(112, 350)
(650, 1882)
(650, 1882)
(650, 1882)
(650, 1882)
(650, 1882)
(650, 1882)
(650, 1882)
(650, 1882)
(100, 361)
(100, 361)
(100, 361)
(100, 361)
(100, 361)
(100, 361)
(100, 361)
(100, 361)
(900, 1500)
(900, 1500)
(900, 1500)
(900, 1500)
(900, 1500)
(900, 1500)
(900, 1500)
(900, 1500)


Extract kNN

In [ ]:
import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

In [ ]:
 # brute_results, result_by_neighbor = general_test(causal_db, effect_db, trueMat, best_gamma=best_gamma,
    #                                                  neighbor_param=[10, 100], lag=lag, pval=pval)

best_gamma = 5
neighbor_param =[2, 5, 10, 100]
lag = 2
pval = 0.05

counter = 0
for each in dataset_names:
    causal = dataset_dict[each]["causaldb"]
    representation = Representation.GRAIL(kernel="SINK", d = 100, gamma = best_gamma)
    trueMat = dataset_dict[each]["truemat"]

    for import_type in to_import:
        effect = dataset_dict[each][import_type]
        n1 = causal.shape[0]
        n2 = effect.shape[0]
        TRAIN_TS, TEST_TS = representation.get_rep_train_test(effect, causal)
        for neighbor_num in neighbor_param:
            if neighbor_num >= n2:
                continue
            neighbors, _, _ = kNN(TRAIN_TS, TEST_TS, method="ED", k=neighbor_num, representation=None, use_exact_rep=True,
                                    pq_method=None) #changed pq

            exact_neighbors, _, _ = kNN(effect, causal, method="SINK", k=neighbor_num, representation=None, gamma_val=best_gamma)

            knn_map_accuracy = MAP(exact_neighbors, neighbors)
            knn_recall_accuracy = avg_recall_measure(exact_neighbors, neighbors)

            print(knn_recall_accuracy, knn_map_accuracy)
            var_names =  np.arange(len(noisydata[3]))

            #repeat below for GRAIL
            t = time()
            for i in range(n1):
                for j in neighbors[i]:
                    df = pp.DataFrame(sample_set_data, datatime = np.arange(len(noisydata[3])),var_names=var_names)
                    # tp.plot_timeseries(df)
                    parcorr = ParCorr(significance='analytic')
                    pcmci = PCMCI(
                        dataframe=df, 
                        cond_ind_test=parcorr,
                        verbosity=1)
                    # correlations = pcmci.get_lagged_dependencies(tau_max=20, val_only=True)['val_matrix']
                    # lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                    #                             'x_base':5, 'y_base':.5}); plt.show()
                    pcmci.verbosity = 1
                    results = pcmci.run_pcmci(tau_max=8, pc_alpha=None)
                    print("p-values")
                    print (results['p_matrix'].round(3))
                    print("MCI partial correlations")
                    print (results['val_matrix'].round(2))
                    #plotting
                    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=8, fdr_method='fdr_bh')
                    pcmci.print_significant_links(
                            p_matrix = results['p_matrix'], 
                            q_matrix = q_matrix,
                            val_matrix = results['val_matrix'],
                            alpha_level = 0.01)
                    link_matrix = pcmci.return_significant_links(pq_matrix=q_matrix,
                        val_matrix=results['val_matrix'], alpha_level=0.01)['link_matrix']
                    #checking results
                    check_results = check_with_original(trueMat, results['val_matrix'].round(2))
                    #do for exact neighbors
                    #do for GRAIL neighbors
                            #get pairs, call PCMCI, get the matrix back, p-values
                            grailMat[i,j] = granger_causality(effect[j], causal[i], lag, pval=pval)
                            #repeat for PCMCI-plus
            prunedtime = time() - t

            #saving results, dump into json?
            result_by_neighbor[neighbor_num] = {'precision' : check_results[0], 'recall' : check_results[1],
                                            'fscore' : check_results[2], 'runtime' : prunedtime,'map' : knn_map_accuracy,
                                            'knn_recall' : knn_recall_accuracy}

Extract kNN with GRAIL

In [ ]:
#representation = Representation.GRAIL(kernel="SINK", d = 100, gamma = best_gamma)
#TRAIN_TS, TEST_TS = representation.get_rep_train_test(Effect_TS, Cause_TS)

Checking

In [ ]:
    # brute_res = check_with_original(trueMat, bruteMat)
    # brute_results = {'precision' : brute_res[0], 'recall' : brute_res[1],
    #                                         'fscore' : brute_res[2], 'runtime' : bruteTime}

1: PCMCI

In [ ]:
import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

Initialize PCMCI Object

In [ ]:
df = pp.DataFrame(sample_set_data, datatime = np.arange(len(noisydata[3])),var_names=var_names)
tp.plot_timeseries(df)

In [ ]:
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=df, 
    cond_ind_test=parcorr,
    verbosity=1)

In [ ]:
correlations = pcmci.get_lagged_dependencies(tau_max=20, val_only=True)['val_matrix']
lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                                    'x_base':5, 'y_base':.5}); plt.show()

Printing PCMCI p-values, correlations

In [ ]:
print("p-values")
print (results['p_matrix'].round(3))
print("MCI partial correlations")
print (results['val_matrix'].round(2))

In [ ]:
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=8, fdr_method='fdr_bh')
pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        alpha_level = 0.01)

Plotting PCMCI results

In [ ]:
link_matrix = pcmci.return_significant_links(pq_matrix=q_matrix,
                        val_matrix=results['val_matrix'], alpha_level=0.01)['link_matrix']

In [ ]:
import importlib
importlib.reload(tp)

# tp.plot_graph(
#     val_matrix=results['val_matrix'],
#     link_matrix=link_matrix,
#     var_names=var_names,
#     link_colorbar_label='cross-MCI',
#     node_colorbar_label='auto-MCI',
#     ); plt.show()

Checking Results

In [ ]:
    # brute_res = check_with_original(trueMat, bruteMat)
    # brute_results = {'precision' : brute_res[0], 'recall' : brute_res[1],
    #                                         'fscore' : brute_res[2], 'runtime' : bruteTime}